# Objetivo
Aprender a utilizar o Folium para visualização de dados em mapas. Para isso usaremos os Dados Públicos de Segurança (SSP-SP) com as ocorrências de veículos subtraídos.

**Origem dos Dados:**

* Os dados vêm de dois sistemas de registro de ocorrências da Polícia Civil de SP:
  **R.D.O. (Registro Digital de Ocorrências)** e **S.P.J. (Sistema de Polícia Judiciária)**.
* O **R.D.O.** foi implementado gradualmente até cobrir todos os municípios em 2010.
  O **S.P.J.** começou a substituir o R.D.O. entre 2022 e 2023.

**Organização das Linhas:**

* **Cada linha da base representa um item do boletim**, podendo ser uma pessoa, um objeto ou uma natureza criminal.
* **Um mesmo boletim pode gerar várias linhas**, dependendo da quantidade de envolvidos.
* Para contar ocorrências únicas, é preciso fazer a **deduplicação**, usando os campos:
  `NOME_DELEGACIA`, `ANO_BO`, `NUM_BO`.

**Localização das Ocorrências:**

* O cidadão pode registrar a ocorrência em qualquer delegacia do Estado, o que gera uma diferença entre:
  **Delegacia de registro** ≠ **Delegacia de circunscrição (local real do fato)**.
* Cerca de **60% das ocorrências são registradas fora da área de circunscrição**.

**Privacidade e Dados Sensíveis:**

* Os dados seguem a **Lei de Acesso à Informação (art. 31)** para proteger a identidade das pessoas.
* **Endereços sensíveis** (ex: residências, escolas, shoppings, etc.) não são divulgados.
  Nesses casos, o campo de localização (`LOGRADOURO`) será preenchido com a frase:
  **“VEDAÇÃO DA DIVULGAÇÃO DOS DADOS RELATIVOS”**.

Fonte: https://www.ssp.sp.gov.br/estatistica/consultas

# Bibliotecas

In [ ]:
import geopandas as gpd
import pandas as pd 
import folium
import matplotlib.pyplot as plt
import base64

from io import BytesIO
from folium.plugins import MarkerCluster, HeatMap, HeatMapWithTime
import branca.colormap as cm

In [ ]:
def grafico_base64(cod_mun, df_munic_time):
    dados = df_munic_time[df_munic_time["cod_municipio"] == cod_mun]
    if dados.empty:
        return ""
    
    plt.figure(figsize=(3, 2))
    plt.plot(dados["data_ocorrencia_bo"], dados["tx_incidentes_100k"], marker="o")

    # Adiciona o valor de cada ponto no gráfico
    for x, y in zip(dados["data_ocorrencia_bo"], dados["tx_incidentes_100k"]):
        plt.text(x, y, f' {y:.1f}', ha='left', va='center', fontsize=7)

    # Remove os valores (rótulos) do eixo Y
    plt.yticks([])
    plt.ylabel("") # Remove o título do eixo Y

    ax = plt.gca() # Obtém o objeto Axes atual
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)

    plt.xticks(rotation=45, fontsize=7)
    plt.title("Tx Incidentes ao longo do ano")
    plt.tight_layout()
    
    buf = BytesIO()
    plt.savefig(buf, format="png")
    plt.close()
    
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode("utf-8")
    html = f'<img src="data:image/png;base64,{img_base64}">'
    return html

# Dados

In [ ]:
# Constantes
grupo_veiculos = {
    "Automovel": "Veículos Leves",
    "Caminhonete": "Veículos Leves",
    "Camioneta": "Veículos Leves",
    "Utilitário": "Veículos Leves",
    "Quadriciclo": "Veículos Leves",
    "Motociclo": "Veículos de Duas/Três Rodas",
    "Motoneta": "Veículos de Duas/Três Rodas",
    "Ciclomoto": "Veículos de Duas/Três Rodas",
    "Triciclo": "Veículos de Duas/Três Rodas",
    "Bicicleta": "Veículos de Duas/Três Rodas",
    "Onibus": "Transporte de Passageiros",
    "Micro-onibus": "Transporte de Passageiros",
    "Caminhão": "Veículos de Carga",
    "Caminhão trator": "Veículos de Carga",
    "Semi-reboque": "Veículos de Carga",
    "Reboque": "Veículos de Carga",
    "Trator rodas": "Maquinário e Especiais",
    "Trator esteiras": "Maquinário e Especiais",
    "Trator misto": "Maquinário e Especiais",
    "Motor casa": "Maquinário e Especiais",
    "Inexistente": "Dados Inválidos",
    "Não informado": "Dados Inválidos"
}

grupo_veiculos_cor = {
    'Dados Inválidos': 'gray',
    'Maquinário e Especiais': 'beige',
    'Transporte de Passageiros': 'cadetblue',
    'Veículos Leves': 'blue',
    'Veículos de Carga': 'darkblue',
    'Veículos de Duas/Três Rodas': 'lightblue'
}

cores_ocorrencia = {
    "Localizado / Entregue": "lightblue",
    "Roubado": "darkblue",
    "Furtado": "cadetblue",
}

In [ ]:
df = pd.read_csv("data/processed/ocorrencias_2025_1sem.csv", dtype={"cod_municipio": str})
geo_sp = gpd.read_file("data/processed/SP_Municipios_2024.geojson")

In [ ]:
df["grupo_veiculo"] = df["descr_tipo"].map(grupo_veiculos)
coordenadas_sp = [df["latitude"].mean(), df["longitude"].mean()]

print((df.shape, geo_sp.shape))
print(df.isnull().sum() / df.shape[0] * 100)

# remoção dos nulos faltantes
df = df.dropna(subset=["latitude", "longitude"])

# Dataviz com Mapas

## 1 - Visualizar a distribuição de veículos subtraídos

### Folium Map

In [ ]:
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=8,
                     tiles="cartodb positron")  

mapa_sp

### Folium Marker

#### Markers

In [ ]:
def add_markers_folium(data, map):
    folium.Marker(
        location=[data["latitude"], data["longitude"]],
    ).add_to(map)
    
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=6,
                     zoom_control=False,
                     tiles="cartodb positron")  

df[(df["data_ocorrencia_bo"] == "2025-06-30")].apply(add_markers_folium, map=mapa_sp, axis=1)

mapa_sp

#### Icon, Tooltip e Popup 

In [ ]:
def add_markers_folium(data, map, cores):
    folium.Marker(
        location=[data["latitude"], data["longitude"]],
        tooltip=data["grupo_veiculo"],
        icon=folium.Icon(color=cores.get(data["grupo_veiculo"], "gray")),
        popup=data["descr_tipo"],
    ).add_to(map)

coordenadas_sp = [df["latitude"].mean(), df["longitude"].mean()]
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=8,
                     zoom_control=False,                  
                     tiles="cartodb positron")  

df[(df["data_ocorrencia_bo"] == "2025-06-30")].apply(add_markers_folium, map=mapa_sp, cores=grupo_veiculos_cor, axis=1)

mapa_sp

#### Clusters

In [ ]:
# função JS que replica o estilo original do Leaflet.MarkerCluster, mas troca a cor
icon_js_template = """
function(cluster) {
    var count = cluster.getChildCount();
    
    // Define as cores para o círculo INTERNO (fundo) e EXTERNO (borda)
    var corFundo = '#434e6c';    // Cor principal, mais escura
    var corBorda = '#6f7a99';    // Cor da "borda", mais clara
    
    if (count > 500) { 
        corFundo = '#bcae6c'; 
        corBorda = '#e0d5a0';
    } else if (count > 100) { 
        corFundo = '#7d7c78';
        corBorda = '#adaaa4';
    }

    // --- Montagem do HTML com dois DIVs ---
    
    // Estilo do círculo externo (a "borda")
    // Ele usa display:flex para centralizar o seu filho (o círculo interno)
    var outer_div_style = 'background-color:' + corBorda + '; width:100%; height:100%; border-radius:50%; display:flex; align-items:center; justify-content:center; font-weight:bold; color:white;';
    
    // Estilo do círculo interno (o "fundo")
    // Adicionamos 'margin:0;' para anular qualquer estilo padrão e garantir a centralização
    var inner_div_style = 'background-color:' + corFundo + '; width:75%; height:75%; border-radius:50%; display:flex; align-items:center; justify-content:center; margin:0;';

    // Juntamos tudo para formar o ícone com os dois círculos
    var html = '<div style="' + outer_div_style + '">' +
                 '<div style="' + inner_div_style + '">' +
                   '<span>' + count + '</span>' +
                 '</div>' +
               '</div>';

    return new L.DivIcon({
        html: html,
        className: 'marker-cluster',
        iconSize: new L.Point(40, 40)
    });
}
"""

In [ ]:
def add_markers_folium_group(data, map, cores_tipo):
    folium.Marker(
        location=[data["latitude"], data["longitude"]],
        tooltip=data["descr_ocorrencia_veiculo"],
        icon=folium.Icon(color=cores_tipo.get(data["descr_ocorrencia_veiculo"], "gray")),
        popup=data["grupo_veiculo"],
    ).add_to(map)

# centro do mapa
coordenadas_sp = [df["latitude"].mean(), df["longitude"].mean()]
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=8,
                     zoom_control=False,  
                     tiles="cartodb positron")  

marker_cluster = MarkerCluster(icon_create_function=icon_js_template).add_to(mapa_sp)

df.apply(add_markers_folium_group, map=marker_cluster, cores_tipo=cores_ocorrencia, axis=1)

mapa_sp

#### Filters

In [ ]:
def add_markers_folium_group(data, map, cores_tipo):
    folium.Marker(
        location=[data["latitude"], data["longitude"]],
        tooltip=data["descr_ocorrencia_veiculo"],
        icon=folium.Icon(color=cores_tipo.get(data["descr_ocorrencia_veiculo"], "gray")),
        popup=data["grupo_veiculo"],
    ).add_to(map)

# centro do mapa
coordenadas_sp = [df["latitude"].mean(), df["longitude"].mean()]
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=8,
                     zoom_control=False,  
                     tiles="cartodb positron")  

clusters_por_grupo = {}
for grupo in set(grupo_veiculos.values()):
    fg = folium.FeatureGroup(name=grupo)
    mc = MarkerCluster(icon_create_function=icon_js_template).add_to(fg)
    fg.add_to(mapa_sp)                
    clusters_por_grupo[grupo] = mc

mapeamento_grupos = {tipo: clusters_por_grupo[grupo] for tipo, grupo in grupo_veiculos.items()}
df.apply(
    lambda x: add_markers_folium_group(
        x,
        map=mapeamento_grupos.get(x["descr_tipo"], clusters_por_grupo.get("Dados Inválidos")),
        cores_tipo=cores_ocorrencia
    ),
    axis=1
)

folium.LayerControl().add_to(mapa_sp)

mapa_sp

## 2 - Visualizar os locais com maior incidência de assaltos

In [ ]:
gradiente_customizado =  {  
                            .4: "#230864", 
                            .6: "#5a3898", 
                            .7: "#ca9efc", 
                            .8: "#f8c871", 
                             1: "#ffec93"
                        }

# https://gka.github.io/palettes
# https://davidmathlogic.com/colorblind/

### Heatmap

In [ ]:
df_incidente = (
    df.groupby(["nome_delegacia", "num_bo"])
      .agg(
            total_veiculos_assaltados=("descr_ocorrencia_veiculo", "count"),
            tipos_veiculos_assaltados=("descr_tipo", lambda x: x.nunique()),
            latitude=("latitude", "first"),
            longitude=("longitude", "first"),
      )
      .reset_index()
)

In [ ]:
coordenadas_sp = [df_incidente["latitude"].mean(), df_incidente["longitude"].mean()]
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=8,
                     zoom_control=False,  
                     tiles="cartodb positron")  

HeatMap(data=df_incidente[["latitude", "longitude"]], 
        radius=10,
        gradient=gradiente_customizado,
        min_opacity=0.5).add_to(mapa_sp)

mapa_sp

### HeatmapWithTime

In [ ]:
df_incidente = (
    df.groupby(["data_ocorrencia_bo", "nome_delegacia", "num_bo"])
      .agg(
            total_veiculos_assaltados=("descr_ocorrencia_veiculo", "count"),
            tipos_veiculos_assaltados=("descr_tipo", lambda x: x.nunique()),
            latitude=("latitude", "first"),
            longitude=("longitude", "first"),
      )
      .reset_index()
)

df_incidente["data_ocorrencia_bo"] = pd.to_datetime(df_incidente["data_ocorrencia_bo"])
df_incidente = df_incidente.sort_values("data_ocorrencia_bo")

heat_data = [
    df_incidente[df_incidente["data_ocorrencia_bo"] == data][["latitude", "longitude"]].values.tolist()
    for data in df_incidente["data_ocorrencia_bo"].unique()
]

coordenadas_sp = [df_incidente["latitude"].mean(), df_incidente["longitude"].mean()]
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=8,
                     zoom_control=False,
                     tiles="cartodb positron")  

HeatMapWithTime(
    heat_data,
    index=[str(d.date()) for d in df_incidente["data_ocorrencia_bo"].unique()],
    radius=8,
    blur=1,
    min_opacity=0.5,
    gradient=gradiente_customizado
).add_to(mapa_sp)

mapa_sp

## 3 - Visualizar a taxa de ocorrências por município

### GeoJson

In [ ]:
df_municipio = (
    df.groupby(["cod_municipio"], as_index=False)
      .agg(incidentes=("num_bo", "nunique"),
           area_km2=("area_km2", "first"),
           populacao_estimada=("populacao_estimada", "first"))
)

df_municipio["tx_incidentes_100k"] = (df_municipio["incidentes"] / df_municipio["populacao_estimada"]) * 100000

geo_sp = geo_sp.merge(df_municipio[["cod_municipio", "tx_incidentes_100k"]].rename(columns={"cod_municipio": "CD_MUN"}),
                      on="CD_MUN",
                      how="left")

In [ ]:
geo_sp.head()

In [ ]:
paleta_personalizada = ['#c3e3ff', '#a3c4fe', '#87a8e8', '#6b8ecc', '#4f74b0', '#325c95', '#0b457b', '#002c5f', '#001544']
colormap_personalizado = cm.LinearColormap(colors=paleta_personalizada)

colormap = colormap_personalizado.scale(
    df_municipio["tx_incidentes_100k"].min(),
    df_municipio["tx_incidentes_100k"].max()
)

colormap.caption = "Taxa de Incidentes por Município (por 100k habitantes)"
colormap

In [ ]:
coordenadas_sp = [df["latitude"].mean(), df["longitude"].mean()]
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=8,
                     zoom_control=False,  
                     tiles="cartodb positron")  

incidentes_munic = dict(zip(df_municipio["cod_municipio"], df_municipio["tx_incidentes_100k"]))

color_dict = {key: colormap(incidentes_munic[key]) for key in incidentes_munic.keys()}

# Adiciona o GeoJson ao mapa
folium.GeoJson(
    geo_sp,
    style_function=lambda feature: {
        "fillColor": color_dict.get(feature["properties"]["CD_MUN"], "lightgray"),
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        "fillOpacity": 0.5,
    },
    tooltip=folium.GeoJsonTooltip(fields=["NM_MUN", "tx_incidentes_100k"], 
                                  aliases=["Município:", "Tx Incidentes (100k)"]),
    highlight_function=lambda feature: {
        "fillColor": (
            "#FFCE77"
        ),
    },
).add_to(mapa_sp)

colormap.add_to(mapa_sp)

mapa_sp

### Geojson + Tooltip

In [ ]:
# Calcula a série temporal da taxa de incidentes por município
df_munic_time = (
    df.groupby(["cod_municipio", df["data_ocorrencia_bo"].str[:4] + "-" + df["data_ocorrencia_bo"].str[5:7]])
      .agg(incidentes=("num_bo", "nunique"), populacao=("populacao_estimada", "first"))
      .reset_index()
)
df_munic_time["tx_incidentes_100k"] = (df_munic_time["incidentes"] / df_munic_time["populacao"]) * 100000

# Adiciona coluna com gráfico base64 em geo_sp
geo_sp["grafico_html"] = geo_sp["CD_MUN"].apply(grafico_base64, df_munic_time=df_munic_time)

coordenadas_sp = [df["latitude"].mean(), df["longitude"].mean()]
mapa_sp = folium.Map(location=coordenadas_sp, 
                     zoom_start=8,
                     zoom_control=False,  
                     tiles="cartodb positron")  

incidentes_munic = dict(zip(df_municipio["cod_municipio"], df_municipio["tx_incidentes_100k"]))
color_dict = {key: colormap(incidentes_munic[key]) for key in incidentes_munic.keys()}

folium.GeoJson(
    geo_sp,
    style_function=lambda feature: {
        "fillColor": color_dict.get(feature["properties"]["CD_MUN"], "lightgray"),
        "color": "black",
        "weight": 1,
        "dashArray": "5, 5",
        "fillOpacity": 0.4,
    },
    tooltip=folium.GeoJsonTooltip(fields=["NM_MUN", "tx_incidentes_100k"], 
                                  aliases=["Município:", "Tx Incidentes (100k)"]),
    highlight_function=lambda feature: {"fillColor": "#FFCE77"},
    popup=folium.GeoJsonPopup(
        fields=["grafico_html"],
        aliases=[""],
        labels=True,
        max_width=250
    )
).add_to(mapa_sp)

colormap.add_to(mapa_sp)

mapa_sp